In [54]:
import os, json
import numpy as np
import pandas as pd
from resultsCollect import regressionLoss

In [55]:
features = {
    'ccpp': 'air temperature, ambient pressure, relative humidity, exhaust vacuum',
    'servo': 'motor, screw, voltage, power gain',
    'insurance': 'age, gender, smoke/not smoke, body mass index',
    'student': 'freetime, study time, absenses'
}

In [60]:
print('-'*82)
print('&   dataset  & Percentage of training samples &   w/o context   &     w context  &')
for data in ['ccpp', 'servo', 'insurance', 'student']:
    # print('-'*82)
    for pc in ['20', '40', '60', '80', 'full']:
            try:
                rae_list_context, rae_list = np.zeros(3), np.zeros(3)
                rae_list_context_gptj, rae_list_gptj = np.zeros(3), np.zeros(3)
                for sim_idx in range(1,4):
                    file_context = 'results/data_%d/%s_fn_%s_all.json' % (sim_idx, data, pc)
                    file = 'results/data_%d/%s_%s_all.json' % (sim_idx, data, pc)
                    with open(file, 'r') as f:
                        data_json = json.load(f)
                    with open(file_context, 'r') as f:
                        data_json_context = json.load(f)

                    rae_list_gptj[sim_idx-1] = regressionLoss(data_json['gptj_test_y'], data_json['test_y'], 'RAE', True)[0]
                    rae_list_context_gptj[sim_idx-1] = regressionLoss(data_json_context['gptj_test_y'], data_json['test_y'], 'RAE', True)[0]
                    rae_list[sim_idx-1] = regressionLoss(data_json['gpt3_test_y'], data_json['test_y'], 'RAE', True)[0]
                    rae_list_context[sim_idx-1] = regressionLoss(data_json_context['gpt3_test_y'], data_json['test_y'], 'RAE', True)[0]
                pc_ = 100 if pc == 'full' else int(pc)
                if rae_list.mean() < rae_list_context.mean():
                    mid = r' \textbf{%.2f $\pm$%.2f} & %.2f $\pm$%.2f ' % (rae_list.mean(), rae_list.std(), rae_list_context.mean(), rae_list_context.std())
                else:
                    mid = r' %.2f $\pm$%.2f & \textbf{%.2f $\pm$%.2f} ' % (rae_list.mean(), rae_list.std(), rae_list_context.mean(), rae_list_context.std())
                    
                if rae_list_gptj.mean() < rae_list_context_gptj.mean():
                    mid_gptj = r' \textbf{%.2f $\pm$%.2f} & %.2f $\pm$%.2f ' % (rae_list_gptj.mean(), rae_list_gptj.std(), rae_list_context_gptj.mean(), rae_list_context_gptj.std())
                else:
                    mid_gptj = r' %.2f $\pm$%.2f & \textbf{%.2f $\pm$%.2f} ' % (rae_list_gptj.mean(), rae_list_gptj.std(), rae_list_context_gptj.mean(), rae_list_context_gptj.std())
                    
                if pc == '20':
                    row = r"\multirow{5}{*}{%s} & \multirow{5}{2cm}{%s} & %s & %s & %s \\" % (data, features[data], pc_/100, mid, mid_gptj)
                else:
                    row = r"& & %s%s & %s & %s \\" % ( pc_/100, ' '*(30-len(pc)), mid, mid_gptj)
                if pc == 'full' and data !='student':
                    row += '\midrule'
                print(row)
            except:
                pass

print('-'*82)

----------------------------------------------------------------------------------
&   dataset  & Percentage of training samples &   w/o context   &     w context  &
\multirow{5}{*}{ccpp} & \multirow{5}{2cm}{air temperature, ambient pressure, relative humidity, exhaust vacuum} & 0.2 &  0.26 $\pm$0.00 & \textbf{0.25 $\pm$0.00}  &  \textbf{0.33 $\pm$0.02} & 0.37 $\pm$0.04  \\
& & 0.4                             &  \textbf{0.24 $\pm$0.00} & 0.24 $\pm$0.00  &  \textbf{0.36 $\pm$0.01} & 0.36 $\pm$0.03  \\
& & 0.6                             &  0.23 $\pm$0.02 & \textbf{0.22 $\pm$0.00}  &  \textbf{0.32 $\pm$0.00} & 0.33 $\pm$0.00  \\
& & 0.8                             &  0.23 $\pm$0.00 & \textbf{0.23 $\pm$0.01}  &  \textbf{0.27 $\pm$0.00} & 0.31 $\pm$0.06  \\
& & 1.0                           &  \textbf{0.18 $\pm$0.01} & 0.21 $\pm$0.00  &  \textbf{0.24 $\pm$0.01} & 0.31 $\pm$0.00  \\\midrule
\multirow{5}{*}{servo} & \multirow{5}{2cm}{motor, screw, voltage, power gain} & 0.2 &  0.81 $\pm$0.01

In [15]:
import pandas as pd
import sys, os, openai, json
import numpy as np
sys.path.insert(1, '../utils')
from GPT3FineTuner import GPT3FineTuner
from functools import partial

In [23]:
data = 'diamond'
pc = '20'
train_df = pd.read_csv("data/%s/%s_train_%s.csv" % (data, data.lower(),pc))
valid_df = pd.read_csv("data/%s/%s_valid.csv" % (data, data.lower()))
test_df = pd.read_csv("data/%s/%s_test.csv" % (data, data.lower()))

config = {'model_type':'ada',"num_epochs":10,"batch_size":5, "lr": [0.05]}

cols = train_df.columns.tolist()
cols[-1] = 'y'
# cols[:-1] = list(range(len(cols) - 1))
train_df.columns = cols
valid_df.columns = cols
test_df.columns = cols

In [24]:
valid_mean = train_df['y'].mean()

In [8]:
y_valid_outputs_,len_valid_valid_y_, rmse_, rmse_woo_ = [], [], [], []

In [9]:
openai.api_key = '[REPLACE IT WITH YOUR OPENAI KEY]'

In [10]:
ft_info = openai.FineTune.retrieve(id='ft-Ajei2iBwGdXTEsPMSbOcDtLr')

In [12]:
ft_model = ft_info['fine_tuned_model']

In [13]:
valid_prompts = [
"When we have x1=1.0000, x2=1.0000, x3=5.0000, x4=4.0000, what should be the y value?###",
"When we have x1=5.0000, x2=2.0000, x3=4.0000, x4=3.0000, what should be the y value?###",
"When we have x1=2.0000, x2=5.0000, x3=4.0000, x4=2.0000, what should be the y value?###"
]

In [18]:
def query(prompt,model,valid_temperature=0.75,valid_mean = 0):
    load_flag = True
    while(load_flag):
        try:
            output =  openai.Completion.create(model = model,prompt = prompt, temperature=0)['choices'][0]['text']
            load_flag = False
        except Exception as e:
            print("%s" % e)
            load_flag = True
            time.sleep(10)
    try:
        return float(output.split('@@@')[0])
    except:
        load_flag = False
        for _ in range(5):
            try:
                output =  openai.Completion.create(model = model,prompt = prompt, temperature=valid_temperature)['choices'][0]['text']
                load_flag = False
            except Exception as e:
                print("%s" % e)
                load_flag = True
                time.sleep(10)

            try:
                return float(output.split('@@@')[0])
            except:
                pass
    return valid_mean
y_name = 'y'

In [25]:
y_valid_outputs = list(map(partial(query, model = ft_model, valid_mean = valid_mean, valid_temperature = 0.75), valid_prompts))
y_valid_outputs_.append(y_valid_outputs)

valid_valid_y = [valid_df[y_name][i] for i in range(len(y_valid_outputs)) if y_valid_outputs[i] != None]
valid_valid_y_outputs = [y_valid_outputs[i] for i in range(len(y_valid_outputs)) if y_valid_outputs[i] != None]

KeyError: 0